In [ ]:
import pandas as pd
import numpy as np

from zipline import run_algorithm
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import AverageDollarVolume, CustomFactor, Returns
from zipline.api import (
    symbol,
    attach_pipeline,
    calendars,
    pipeline_output,
    date_rules,
    time_rules,
    get_datetime,
    set_commission,
    set_slippage,
    record,
    order_target_percent,
    get_open_orders,
    schedule_function,
    set_commission,
    set_slippage,
    set_benchmark
)
from zipline.finance import commission, slippage
import pyfolio as pf
from zipline.data import bundles

### Trade Settings

In [ ]:
N_LONGS = N_SHORTS = 50
DOLLAR_VOLUME = 500

In [ ]:
import os
import dotenv
from dotenv  import find_dotenv, load_dotenv
load_dotenv(find_dotenv())
NASDAQ_API_KEY = os.getenv("NASDAQ_KEY")

In [ ]:
os.environ["QUANDL_API_KEY"] = NASDAQ_API_KEY
bundle = "quandl"
bundles.ingest(bundle)

In [ ]:
class MomentumFactor(CustomFactor):
    inputs = [USEquityPricing.close, Returns(window_length=126)]
    window_length = 252

    def compute(self, today, assets, out, prices, returns):
        out[:] = (
            (prices[-21] - prices[-252]) / prices[-252]
            - (prices[-1] - prices[-21]) / prices[-21]
        ) / np.nanstd(returns, axis=0)

In [ ]:
def make_pipeline():
    momentum = MomentumFactor()
    dollar_volume = AverageDollarVolume(window_length=21)
    return Pipeline(
        columns={
            "factor": momentum,
            "longs": momentum.top(N_LONGS),
            "shorts": momentum.bottom(N_SHORTS),
            "ranking": momentum.rank(),
        },
        screen=dollar_volume.top(DOLLAR_VOLUME),
    )

In [ ]:
def before_trading_start(context, data):
    context.factor_data = pipeline_output("factor_pipeline")

In [ ]:
def rebalance(context, data):
    factor_data = context.factor_data
    record(factor_data=factor_data.ranking)

    assets = factor_data.index
    record(prices=data.current(assets, "price"))

    longs = assets[factor_data.longs]
    shorts = assets[factor_data.shorts]
    divest = set(context.portfolio.positions.keys()) - set(longs.union(shorts))
    
    print(
        f"{get_datetime().date()} Longs: {len(longs)} Shorts: {len(shorts)} Value:{context.portfolio.portfolio_value}"
    )

    exec_trades(
        data, 
        assets=divest, 
        target_percent=0
    )
    exec_trades(
        data, 
        assets=longs, 
        target_percent=1 / N_LONGS
    )
    exec_trades(
        data, 
        assets=shorts, 
        target_percent=-1 / N_SHORTS
    )

In [ ]:
def initialize(context):
    attach_pipeline(make_pipeline(), "factor_pipeline")
    schedule_function(
        rebalance,
        date_rules.week_start(),
        time_rules.market_open(),
        calendar=calendars.US_EQUITIES,
    )
    
    # Set up the commission model to charge us per share and a volume slippage model
    set_commission(
        us_equities=commission.PerShare(
            cost=0.005,
            min_trade_cost=2.0
        )
    )
    set_slippage(
        us_equities=slippage.VolumeShareSlippage(
            volume_limit=0.0025, 
            price_impact=0.01
        )
    )
    # set_benchmark(symbol("SPY"))

In [ ]:
def exec_trades(data, assets, target_percent):
    for asset in assets:
        if data.can_trade(asset) and not get_open_orders(asset):
            order_target_percent(asset, target_percent)

In [ ]:
def analyze(context, perf):
    perf.portfolio_value.plot()

In [ ]:
start = pd.Timestamp("2016-01-01")
end = pd.Timestamp("2017-12-31")

import pandas_datareader.data as web
sp500 = web.DataReader('SP500', 'fred', start, end).SP500
benchmark_returns = sp500.pct_change()

In [ ]:
perf = run_algorithm(
    start=pd.Timestamp("2016-01-01"),
    end=pd.Timestamp("2017-12-31"),
    initialize=initialize,
    before_trading_start=before_trading_start,
    capital_base=100_000,
    # bundle="quotemedia",
    bundle=bundle
)

In [ ]:
returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)

In [ ]:
pf.create_full_tear_sheet(returns, positions, transactions)